In [ ]:
import oat_python as oat

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import sklearn
import sklearn.metrics
from sklearn.neighbors import NearestNeighbors
import itertools

# Homology of a Dowker complex

In this example we will
- generate a point cloud
- generate a collection of subsets (some would call this a *cover*, others would call it a *hypergraph*)
- compute the homology of the associated Dowker complex
- analyze cycle representatives
- plot cycle representatives

## Generate a point cloud

In [ ]:
cloud               =   oat.point_cloud.annulus(npoints=60, rad0=1, rad1=2.5, random_seed=0)

#   PLOT
trace               =   go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers")
fig                 =   go.Figure([trace])
fig.update_layout( title=dict(text="Circle with noise"), width=800, height=800 )
fig.show()

## Choose a cover

In [ ]:
radius_neighbor =   1.2; # hyperedges will be neighborhoods of vertices, with this radius
radius_net      =   1.0; # we'll make epsilon net with this value of epsilon

#   COMPUTE AN EPSILON NET
net, _          =   oat.dissimilarity.farthest_point_with_cloud(
                        cloud   =   cloud,
                        epsilon =   radius_net,
                    )

#   PLOT THE COVER

data            =   []
data.append( go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers", name="Cloud", showlegend=True)                                               )
data.append( go.Scatter(x=cloud[net,0],y=cloud[net,1], mode="markers", marker=dict(symbol="triangle-up", size=10), name="Net", showlegend=True) )


for counter, v in enumerate(net):
    point = cloud[v]
    trace = oat.plot.ball_2d( point[0], point[1], radius=radius_neighbor, npoints=100 )
    trace.update( opacity=0.2, name=f"Cover {counter}" )
    data.append(trace)

fig = go.Figure(data)
fig.update_layout( title=dict(text="Hyperedges"), width=800, height=800 )
fig.show()

## Compute homology

We'll compute the homology of
- the dual hypergraph; that is, the hypergraph where vertices are balls, and for each vertex `v` we have a hyperedge that contains every ball to which `v` belongs
- equivalently, the witness complex where every point is a witness and net points are landmarks

The homology solver only accepts hypergraphs represented by a list of lists of integers, currently.  If your hypergraph has a different format (e.g., if vertices are strings), then you can use some built-in tools to help translate back and forth between this format and list-of-list format; see the [rbs_reduced](rbs_reduced.ipynb) notebook for examples.

In [ ]:
#   FORMAT THE COVER AS A FAMILY OF SORTED LISTS

# data structure holding the whole cloud
net_wrapper         =   NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit( cloud[net] ) 
# witness complex where all points are witnesses, and net points are landmarks
cover               =   [ sorted(list(x)) for x in net_wrapper.radius_neighbors( cloud, radius=radius_neighbor, return_distance=False, ) ]    

#   FACTOR THE BOUNDARY MATRIX

factored            =   oat.rust.FactoredBoundaryMatrixDowker( 
                            dowker_simplices            =   cover, 
                            max_homology_dimension      =   2
                        )

#   BETTI STATISTICS

print("\n\nBetti statistics\n")
display( factored.betti() )

#   HOMOLOGY

print("\nHomology\n")
homology    =   factored.homology()
display( homology )

#   CYCLE REPRESENTATIVE

print("\nCycle representative\n")
display( homology["cycle representative"][1] )


## Plot a cycle

In [ ]:
#   DATA FOR THE POINT CLOUD

data            =   []
data.append( go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers", name="Cloud", showlegend=True)                                               )
data.append( go.Scatter(x=cloud[net,0],y=cloud[net,1], mode="markers", marker=dict(symbol="triangle-up", size=10), name="Net", showlegend=True) )

#   DATA FOR THE CYCLE

cycle_num           =   3
cycle               =   homology["cycle representative"][cycle_num]
coo                 =   cloud[net] # coo stands for "coordinate oracle"

for counter, linear_term in cycle.iterrows():
    simplex         =   linear_term["simplex"]
    coefficient     =   linear_term["coefficient"]

    trace           =   oat.plot.edge__trace2d( edge=simplex, coo=coo )
    trace.update( name=f"Edge {simplex}", text=f"simplex {simplex}<br>linear coefficent {coefficient}", opacity=0.7, ) # customize appearance
    data.append(trace) # append to the data group 

#   PLOT

fig = go.Figure( data )
fig.update_layout(title="Coefficients appear in the hover data", height=700,width=800 ) 
fig.show()

# Group simplices into a single legend entry

Sometimes you don't want a separate legend entry for each simplex.  Here's an example that shows how to group them.

In [ ]:
counter = 0
for trace in fig.data: # loop over traces in the figure
    if "Edge" in trace.name: # if the name of the trace contains "simplex"
        trace.update(legendgroup="1") # tag the group with a label; the same label for all simplex traces
        trace.update(name="Cycle edges") # then change the "name" to the text we want to appear in the legend
        if counter!=0: # otherwise
            trace.update(showlegend=False) # remove the trace from the legend; this gaurantees that only one legend entry appears, for all traces
        counter +=1

fig.update_layout(title="Coefficients still apper in the hover data", height=700,width=800,) 
fig.show()        


# Assign the same color to all edges in a given cycle

In [ ]:
#   DATA FOR THE POINT CLOUD

data                    =   []
data.append( go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers", name="Cloud", showlegend=True)                                               )
data.append( go.Scatter(x=cloud[net,0],y=cloud[net,1], mode="markers+text", marker=dict(symbol="triangle-up", size=10), name="Net", showlegend=True, text=[str(x) for x in range(len(net))],  textposition='bottom center',  ) )

#   DATA FOR THE CYCLE

coo                     =   cloud[net] # coo stands for "coordinate oracle"
colors                  =   px.colors.qualitative.Plotly # specifies a (discrete) sequence of colors, represented by a list of strings

for rownum, row in homology.iterrows():
    if row["dimension"] != 1: continue
    cycle_color         =   colors[ rownum % len(colors) ]

    for entrynum, entry in row["cycle representative"].iterrows():
        edge            =   entry["simplex"]
        coefficient     =   entry["coefficient"]

        trace           =   oat.plot.edge__trace2d( edge=edge, coo=coo )
        trace.update( name=f"Cycle {rownum}", text=f"simplex {edge}<br>linear coefficent {coefficient}", opacity=0.7, line=dict(color=cycle_color,)) # customize appearance
        trace.update( legendgroup=rownum) # group edges that belong to the same cycle
        trace.update( showlegend = entrynum==0 )
        data.append(trace) # append to the data group

#   PLOT

fig = go.Figure( data )
fig.update_layout(title="Coefficients still appear in the hover data", height=700,width=800 ) 
fig.show()